Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./whiskey-reviews-dspt4/train.csv')
test = pd.read_csv('./whiskey-reviews-dspt4/test.csv')


print(train.shape, test.shape)

(4087, 3) (1022, 2)


In [2]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [3]:
# Distribution of ratingCategory: 0 (Excellent), 1 (Good), 2 (Poor)
train.ratingCategory.value_counts(normalize=True)

1    0.704918
0    0.279178
2    0.015904
Name: ratingCategory, dtype: float64

In [4]:
# Read a few reviews from the "Excellent" category
pd.set_option('display.max_colwidth', 0)
train[train.ratingCategory == 0].sample(3)

,id,description,ratingCategory
798,3778,"\nDistilled between October and March, matured in American (first-fill and refill) and European oak casks; meant to be served ice-cold. From the freezer, concentrated aromas of pineapple, hints of honey and smoke. At room temperature, less intense fruitiness, more honey, and ginger. A syrupy mouthfeel from the freezer, with soft spices and orchard fruits. The palate is more complex at room temperature, with heathery spice, pepper, apricots, raisins, and milk chocolate. The finish yields cocoa powder and lingering spices. £38",0
1559,3907,"\nLimited by its obvious youth on the nose, but the fresh herb chiffonade character offers intrigue, with distinct sage coupled with some chamomile notes. The palate is a step in the right direction, with dark berries, spices, vanilla, and charred marshmallows, although somewhat light bodied and supple in texture. Young, but sweet and honest.",0
2252,4187,"\nNice color for a 10 year old. Puffing peat off the glass, with briny notes and sweetness: promising. The promise is kept in the mouth, too. It’s all there: peat smoke, brine, good round malt, some fruity notes in the finish, and a nice creamy texture. There’s a bit of astringency, and it’s hot for the proof. Still, a very convincing American iteration of a peated single malt, especially at this age.",0


In [5]:
# Read a few reviews from the "Poor" category
train[train.ratingCategory == 2].sample(3)

,id,description,ratingCategory
2396,5088,"\nOne of the most popular flavored whiskies today is Fireball cinnamon, so it’s no surprise that other cinnamon whiskies are entering the market. This one has a lovely, woody, cinnamon nose that bursts into sweet, blistering cinnamon on the palate. Cinnamon is a natural whisky flavor, but here, rather than complement the underlying whisky, it completely masks it. This is a cinnamon liqueur, and a good one. A fun shooter, perhaps, but it’s barely whisky.",2
2609,5079,"\nArtificial cherry aroma, like cherry PEZ, and barely a hint of bourbon. The taste is not overpoweringly sweet at all, a surprise, and there's a nutty, almost Luxardo-like character in the cherry that would be more interesting if it weren't for the soapy notes and bitter medicinal flavor of the whiskey. A more natural cherry character would net a higher rating.",2
211,5054,"\nNose is sweet corn, with some mintiness to it, and some herbal notes; hot, but not off-putting. Tastes are not as sweet as expected. A bit flat, though the sweet corn does come through toward the end. Again, though, this one is quite hot, which is not surprising; the Hudson Baby Bourbons — which this is the foundation for — have struck me as hot also. Price is per 375 ml.",2


In [9]:
# Import Statements
from sklearn.pipeline import Pipeline
# from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

### Split the Training Set into Train/Validation

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['description'], 
                                                    train['ratingCategory'], 
                                                    test_size=0.2, 
                                                    stratify=train['ratingCategory'],
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3269,) (818,) (3269,) (818,)


### Define Pipeline Components

In [11]:
# Build a Baseline TF-IDF Model
# Create Pipeline Components
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()
# vect = ...
# clf = ...

pipe = Pipeline([('vect', vect), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [13]:
# parameters = {
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(5,10,15,20)
# }

# grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(..., ...)


parameters = {
    'vect__max_df': (0.7, 1.0),
    'vect__min_df': (2, 5, 10),
    'vect__max_features': (5000, 10000),
    'clf__n_estimators': (100, 500),
    'clf__max_depth': (10, 20, None)
}

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   48.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  9.6min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [14]:
# Predictions on test sample
pred = grid_search.predict(X_test)

In [16]:
len(pred)

818

In [17]:
len(test['id'])

1022

In [21]:
y_train.head()

1782    1
3414    1
2744    0
77      1
354     1
Name: ratingCategory, dtype: int64

In [28]:
len(X_train)

3269

In [31]:
len(y_train)

3269

In [30]:
len(X_test)

818

In [29]:
len(y_test)

818

In [22]:
y_test.head()

2368    0
3794    1
1380    1
239     0
436     1
Name: ratingCategory, dtype: int64

In [23]:
X_train.head()

1782    \nVery fragrant, with notes of fresh ripe plum, crisp spice, herbs, caramel, treacle, black pepper, and hints of dark chocolate. Leave the glass alone and the spices really hit their stride. The flavor shifts imperceptibly, covering citrus, vanilla, chocolate, and ripe orchard fruits, generating a slow build-up of spices like a gathering storm: your taste buds have nowhere left to run. Beautifully paced.                                                                             
3414    \nThe good news: This is one of the best Highland Park whiskies I have ever tasted. The bad news: it’s the new release for Travel Retail (formerly Duty Free). It’s lush, well-balanced, and very complex. Well-defined notes of toffee, candied fruit, and roasted nuts are accentuated by background honey, chocolate mousse, and smoke. Here’s the icing on the cake: it’s bottled at 47.5%, which really allows the flavors to shine. If you’re going overseas, consider tracking down a bottle.
2744    \nA sw

In [15]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

ValueError: array length 818 does not match index length 1022

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
subNumber = 0

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD

# Use parameters from RandomSearch in previous section
vect = TfidfVectorizer(stop_words='english', 
                       ngram_range=(1,2),
                       min_df=2, 
                       max_df=0.7,
                       max_features=10000)

# Add dimensionality reduction
svd = TruncatedSVD(algorithm='randomized',
                   n_iter=10)

# Use parameters from RandomSearch in previous section
rfc = RandomForestClassifier(n_estimators=500, 
                             random_state=42)

In [ ]:
params = {
    'svd__n_components': stats.randint(100, 500),
}

In [ ]:
pipe = Pipeline([
    ('vect', vect),      # TF-IDF Vectorizer
    ('svd', svd),        # Truncated SVD Dimensionality Reduction
    ('clf', rfc)         # RandomForest Classifier
])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
# parameters = {
#     'lsi__svd__n_components': [10,100,250],
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(5,10,15,20)
# }

# grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(train.description, train.ratingCategory)
# # grid_search.fit(..., ...)

# Fit
random_search = RandomizedSearchCV(pipe, params, cv=2, n_iter=3, n_jobs=-1, verbose=1)
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_score_


In [ ]:
random_search.best_params_

### Make a Submission File

In [ ]:
# Predictions on test sample
# pred = grid_search.predict(test['description'])
pred = random_search.predict(test['description'])


In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
# Apply to your Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [ ]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [ ]:
X = get_word_vectors(X_train)
len(X) == len(X_train)

In [ ]:
X_test = get_word_vectors(X_test)

In [ ]:
rfc.fit(X, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluate on test data
y_test = rfc.predict(X_test)
accuracy_score(X_test, y_test)

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(16, 2),
                    random_state=1)

In [ ]:
clf.fit(X, y_train)

In [ ]:
# Evaluate on test data
y_test = clf.predict(X_test)
accuracy_score(X_test, y_test)


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = clf.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./whiskey-reviews-dspt4/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?